# Import des données

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("lupin.csv")

# Inspection des données

In [ ]:
df.head()

,Note,Description,key
0,"4,5","Les critiques négatives m’étonnent car, pour m...",0
1,"1,5",Alors ce fameux Lupin Dans L'Ombre D'Arsène. Q...,1
2,"4,5","Bon euh, faut pas déconner. Les gens s'acharne...",2
3,"0,5",C'est dommage parce-que les intentions de la s...,3
4,"0,5",Lupin par Omar SY ? on aurait le droit d'hurle...,4


In [ ]:
df["Description"][6]

"Netflix ne sais pas faire de série françaises , \nC'est un naufrage total . C'est la pire adaptation d'Arsène Lupin , a non c'est vrai ce n'est pas Arsène Lupin mais Assane Diop un copicat lol .\nOmar Sy en fait des tonnes pour sauver le naufrage . L'intrigue  si on peut parler d'intrigue \nC'est téléphoné . Bref a fuir immédiatement\nNetflix commande une seconde saison 02 il vont continuer le navet\nGeorges Descrières est Arsène Lupin et aucun acteur ne lui arrivent à la cheville Il doit se retourner dans sa tombe"

In [ ]:
df.shape
df.isnull().sum()
df["key"].describe()

count    2630.000000
mean        3.796578
std         3.707967
min         0.000000
25%         1.000000
50%         3.000000
75%         4.000000
max        14.000000
Name: key, dtype: float64

# Installation de Spacy French

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download fr_core_news_md

     |████████████████████████████████| 88.6MB 1.4MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp37-none-any.whl size=90338490 sha256=717addf2f6b4769ebaef179f07b89148c09af7b3192723122ce4cafa44fc149d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ac1usbvh/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [ ]:
import spacy
import fr_core_news_md

In [ ]:
nlp = fr_core_news_md.load()

# Création des listes négatives et positives de mots

In [ ]:
stopwords =  nlp.Defaults.stop_words
negatifs_words = ['ne', 'n\'', 'pas', 'plus', 'jamais', 'guère', 'nul', 'na', 'ni', 'mais', 'rien']
print(len(stopwords))
positif_stopwords = [x for x in stopwords if x not in negatifs_words]
print(len(positif_stopwords))

600
591


In [ ]:
#Remplace les virgules par des points
df["Note"] = [x.replace(",",".") for x in df["Note"]]

# Variable Target

In [ ]:
#Construction de la variable Target. Conversion des notes en variables binaires
df["Target"] = df.Note.apply(lambda x: 1 if float(x) > 2.5 else 0)

In [ ]:
df["Negatifs"] = df.Description.apply(lambda x: 0 if x.lower() in negatifs_words else 1)
#df["Positifs"] = [x for x in df["Description"] if x.lower() not in negatifs_words]

In [ ]:
df["Negatifs"][0:5]

0    1
1    1
2    1
3    1
4    1
Name: Negatifs, dtype: int64

# Naïf Bayes avec sklearn

In [ ]:
df["Description"]

0       Les critiques négatives m’étonnent car, pour m...
1       Alors ce fameux Lupin Dans L'Ombre D'Arsène. Q...
2       Bon euh, faut pas déconner. Les gens s'acharne...
3       C'est dommage parce-que les intentions de la s...
4       Lupin par Omar SY ? on aurait le droit d'hurle...
                              ...                        
2625    La série est super, très intriguante avec un p...
2626    Le jeu d'acteurs est si mauvais que ça en devi...
2627    Jeu de l’acteur principal catastrophique c’est...
2628    Moderniser la saga de Leblanc pourquoi pas apr...
2629    Franchement le scénario est tiré ar les cheveu...
Name: Description, Length: 2630, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df2 = df.copy()

In [ ]:
X = df2["Description"]
y = df2["Target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
CV = CountVectorizer()
CV.fit(X)
X_train_v = CV.transform(X_train)
X_test_v = CV.transform(X_test)

In [ ]:
gnb = GaussianNB()
gnb_1 = gnb.fit(X_train_v.toarray(), y_train)#.predict(X_test_v.toarray())

In [ ]:
#Résultats des data train et test
print(f"Score du Train set = {gnb_1.score(X_train_v.toarray(), y_train)}", "\n",\
      f"Score du Test set = {gnb_1.score(X_test_v.toarray(), y_test)}")

Score du Train set = 0.9875067897881586 
 Score du Test set = 0.9391634980988594


# Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
RL = LogisticRegression()
RL_1 = RL.fit(X_train_v, y_train)

In [ ]:
#Résultats des data train et test
print(f"Score du Train set = {RL_1.score(X_train_v, y_train)}", "\n",\
      f"Score du Test set = {RL_1.score(X_test_v, y_test)}")

Score du Train set = 0.9994568169473113 
 Score du Test set = 0.9708491761723701
